In [12]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [13]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.metrics import confusion_matrix

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

<IPython.core.display.Javascript object>

In [14]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age"], axis=1)
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,1,1,211,1,50
1,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,149,1,40
2,27,1,0,0,0,1,1,1,1,1,...,1,1,1,1,0,1,1,168,1,28
3,29,1,1,0,1,1,1,1,1,0,...,1,1,1,0,0,1,0,230,1,37
5,19,1,0,0,0,1,1,0,1,1,...,0,0,1,0,0,0,1,361,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,243,1,24
11239,10,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,197,1,39
11240,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,447,2,33
11241,12,0,0,1,0,0,1,0,1,0,...,1,1,0,0,0,0,0,167,1,24


<IPython.core.display.Javascript object>

In [15]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.635 (+/- 0.019)


<IPython.core.display.Javascript object>

In [16]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
9927,2,1
10631,1,1
5241,1,2
8580,1,1
435,2,2
...,...,...
7400,1,1
9659,1,1
11005,2,1
2517,2,2


,Questions,Male,Female,Other
0,Influence,0.613062,0.608936,-1.221998
1,Modesty,0.263693,0.053508,-0.317201
2,Daring,0.214097,0.268551,-0.482648
3,Confidence,-0.384763,-0.614288,0.999051
4,Ruling,0.555987,-0.010880,-0.545107
5,Responsibility,0.154798,0.002340,-0.157138
6,IntrovExtro,-0.156537,0.232388,-0.075851
7,Success,0.343345,0.345055,-0.688400
8,Humble,0.354416,0.270929,-0.625345
9,Leadership,0.215970,0.307437,-0.523406


<IPython.core.display.Javascript object>

In [17]:
# Using Statsmodels.api
# logit_model = sm.MNLogit(Y, X)
# result = logit_model.fit()
# print(result.summary2())

<IPython.core.display.Javascript object>

In [18]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[1367,  407,    7],
       [ 680,  625,    4],
       [   6,    1,    0]], dtype=int64)

<IPython.core.display.Javascript object>

### Multinomial Log Reg with Age Group

In [19]:
data = pd.read_csv("Data/data_AgeGroup.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age", "Age_Group"], axis=1)
Y = data["Age_Group"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age,Age_Group
0,18,2,2,2,2,1,2,1,2,2,...,1,2,2,2,1,2,211,1,50,2.0
1,6,2,2,2,1,2,2,1,2,1,...,1,2,2,2,2,1,149,1,40,2.0
2,27,1,2,2,1,2,1,2,1,2,...,2,1,1,2,1,2,168,1,28,2.0
3,29,1,1,2,2,2,1,2,1,1,...,2,1,2,2,1,1,230,1,37,2.0
4,6,1,2,1,1,1,2,1,2,1,...,1,2,2,2,0,1,389,1,50,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,1,2,2,2,2,1,243,1,24,1.0
11239,10,2,2,1,1,1,2,1,1,1,...,1,2,1,2,2,1,197,1,39,2.0
11240,6,1,2,2,1,1,2,1,2,1,...,1,2,2,2,2,1,447,2,33,2.0
11241,12,2,2,1,1,1,1,1,1,1,...,2,2,2,2,2,1,167,1,24,1.0


<IPython.core.display.Javascript object>

In [20]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

Mean Accuracy: 0.669 (+/- 0.012)


<IPython.core.display.Javascript object>

In [21]:
model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
1635,1.0,1.0
699,2.0,2.0
5053,3.0,2.0
3407,1.0,1.0
3019,2.0,2.0
...,...,...
9200,2.0,2.0
1516,2.0,2.0
1044,2.0,2.0
7280,2.0,2.0


,Questions,Male,Female,Other
0,Influence,0.189115,0.051849,-0.240965
1,Modesty,-0.047834,0.052557,-0.004724
2,Daring,-0.192066,-0.125797,0.317863
3,Confidence,0.139806,-0.139490,-0.000316
4,Ruling,0.162259,-0.040253,-0.122006
5,Responsibility,-0.304990,-0.070597,0.375587
6,IntrovExtro,0.098435,0.125472,-0.223907
7,Success,-0.432772,-0.077863,0.510636
8,Humble,0.160930,0.110129,-0.271059
9,Leadership,0.132319,0.225218,-0.357537


<IPython.core.display.Javascript object>

In [22]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[ 274,  739,    0],
       [ 219, 1890,    0],
       [   5,   88,    0]], dtype=int64)

<IPython.core.display.Javascript object>